In [1]:
import numpy as np
import pandas as pd

from load import read_csv, to_csv
from utilities import get_indicator, add_aggregate

In [2]:
path = './Data/bt-impute.csv'

df = read_csv(path)
pd.set_option('display.max_columns', None)

print(len(df))
df.head()

53096


,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,SITE_ICD10_O2_3CHAR,MORPH_ICD10_O2,BEHAVIOUR_ICD10_O2,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LINKNUMBER,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS
0,10614165,10501771,2017-01-16,C71,9440.0,3,U,G4,69.0,1,L0401,810501771,R,4,2,W,A,False,False,False,0.0,False,False,0.000,0.00,0,751
1,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,1.0,False,False,1.755,71.75,0,17
2,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,3.0,False,False,1.755,71.75,0,17
3,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,2.0,False,False,1.755,71.75,0,17
4,10614166,10501772,2015-11-10,C71,9440.0,3,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,4.0,False,False,1.755,71.75,0,17


In [3]:
df = df.drop(df[(df.SITE_ICD10_O2_3CHAR != 'C70') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C71') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C72') & 
                    (df.SITE_ICD10_O2_3CHAR != 'D32') &
                    (df.SITE_ICD10_O2_3CHAR != 'D33') &
                    (df.SITE_ICD10_O2_3CHAR != 'D35')].index)
print(df.shape)
df.SITE_ICD10_O2_3CHAR.value_counts()

(53001, 27)


C71    31478
D32    11606
D33     4622
D35     4145
C72      844
C70      306
C13        0
D41        0
D09        0
D37        0
D43        0
C91        0
D47        0
C44        0
C51        0
C54        0
C67        0
C77        0
C84        0
C92        0
D01        0
C80        0
C82        0
C17        0
C20        0
C21        0
C22        0
C23        0
C24        0
C25        0
C43        0
C48        0
C50        0
C53        0
C64        0
C65        0
C73        0
C78        0
C79        0
D03        0
Name: SITE_ICD10_O2_3CHAR, dtype: int64

In [4]:
site_dummies = pd.get_dummies(df.SITE_ICD10_O2_3CHAR, prefix='SITE')
df = df.drop(columns = 'SITE_ICD10_O2_3CHAR')
df = pd.concat([df, site_dummies], axis=1)

# for col in site_dummies.columns:
#     df = get_indicator(df, col, 'PATIENTID',)
    

print(df.shape)

(53001, 66)


In [5]:
# dropping identifying numbers
# this should go last
# df = df.drop(columns = ['TUMOURID',	'PATIENTID'	,'DIAGNOSISDATEBEST','LINKNUMBER'])

In [6]:
patient_ids = df['PATIENTID'].copy()

In [7]:
df.MORPH_ICD10_O2.value_counts().to_csv('./Data/morph.csv', index=True)
# too little variations
df = df.drop(columns='MORPH_ICD10_O2')

In [8]:
df.BEHAVIOUR_ICD10_O2 = df.BEHAVIOUR_ICD10_O2.replace({2: 0, 5: 3, 6: 3, 9: 3})
df = df.drop(df[df.BEHAVIOUR_ICD10_O2 == 1].index)
df.BEHAVIOUR_ICD10_O2.value_counts()

# note that pytorch requires float32 instead of int
# transform elsewhere
df['BENIGN_BEHAVIOUR'] = (df.BEHAVIOUR_ICD10_O2 == 0).astype(int)
df = df.drop(columns= 'BEHAVIOUR_ICD10_O2')
df.head()

,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LINKNUMBER,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,SITE_C13,SITE_C17,SITE_C20,SITE_C21,SITE_C22,SITE_C23,SITE_C24,SITE_C25,SITE_C43,SITE_C48,SITE_C50,SITE_C53,SITE_C64,SITE_C65,SITE_C70,SITE_C71,SITE_C72,SITE_C73,SITE_C78,SITE_C79,SITE_C80,SITE_C82,SITE_C91,SITE_D01,SITE_D09,SITE_D32,SITE_D33,SITE_D35,SITE_D37,SITE_D41,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR
0,10614165,10501771,2017-01-16,U,G4,69.0,1,L0401,810501771,R,4,2,W,A,False,False,False,0.0,False,False,0.000,0.00,0,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10614166,10501772,2015-11-10,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,1.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10614166,10501772,2015-11-10,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,3.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10614166,10501772,2015-11-10,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,2.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10614166,10501772,2015-11-10,U,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,4.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
### delete ###
print(df['PATIENTID'].nunique())
df.groupby('PATIENTID').size().sort_values()

40018


PATIENTID
10501771      1
250018286     1
250018287     1
250018288     1
250018289     1
             ..
210004002    24
210007896    26
210010978    26
250012125    26
210010917    28
Length: 40018, dtype: int64

In [10]:
### delete ###
df.loc[df['PATIENTID']==210010978]

,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LINKNUMBER,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,SITE_C13,SITE_C17,SITE_C20,SITE_C21,SITE_C22,SITE_C23,SITE_C24,SITE_C25,SITE_C43,SITE_C48,SITE_C50,SITE_C53,SITE_C64,SITE_C65,SITE_C70,SITE_C71,SITE_C72,SITE_C73,SITE_C78,SITE_C79,SITE_C80,SITE_C82,SITE_C91,SITE_D01,SITE_D09,SITE_D32,SITE_D33,SITE_D35,SITE_D37,SITE_D41,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR
18728,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,False,True,6.0,False,True,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18729,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,False,False,3.0,False,False,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18730,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,False,False,8.0,True,True,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18731,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,False,False,2.0,False,False,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18732,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,False,False,1.0,False,False,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18733,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,False,True,5.0,False,True,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18734,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,True,False,7.0,True,True,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18735,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,True,False,9.0,True,True,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18736,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,True,False,14.0,True,True,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18737,210010978,210010978,2017-05-16,U,G4,72.0,2,L0201,1010010978,L,4,1,W,D,True,False,False,15.0,True,True,1.75,53.7,0,633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
df = df.drop(columns='STAGE_BEST')
df.head()

,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,GRADE,AGE,SEX,CREG_CODE,LINKNUMBER,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,SITE_C13,SITE_C17,SITE_C20,SITE_C21,SITE_C22,SITE_C23,SITE_C24,SITE_C25,SITE_C43,SITE_C48,SITE_C50,SITE_C53,SITE_C64,SITE_C65,SITE_C70,SITE_C71,SITE_C72,SITE_C73,SITE_C78,SITE_C79,SITE_C80,SITE_C82,SITE_C91,SITE_D01,SITE_D09,SITE_D32,SITE_D33,SITE_D35,SITE_D37,SITE_D41,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR
0,10614165,10501771,2017-01-16,G4,69.0,1,L0401,810501771,R,4,2,W,A,False,False,False,0.0,False,False,0.000,0.00,0,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10614166,10501772,2015-11-10,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,1.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10614166,10501772,2015-11-10,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,3.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10614166,10501772,2015-11-10,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,2.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10614166,10501772,2015-11-10,G4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,4.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
grade_mapper = {'GX' : -1, 'G0' : 0, 'G1' : 1, 'G2' : 2, 'G3' : 3, 'G4' : 4}
df.GRADE = df.GRADE.replace(grade_mapper)
df.head()

,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,GRADE,AGE,SEX,CREG_CODE,LINKNUMBER,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,SITE_C13,SITE_C17,SITE_C20,SITE_C21,SITE_C22,SITE_C23,SITE_C24,SITE_C25,SITE_C43,SITE_C48,SITE_C50,SITE_C53,SITE_C64,SITE_C65,SITE_C70,SITE_C71,SITE_C72,SITE_C73,SITE_C78,SITE_C79,SITE_C80,SITE_C82,SITE_C91,SITE_D01,SITE_D09,SITE_D32,SITE_D33,SITE_D35,SITE_D37,SITE_D41,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR
0,10614165,10501771,2017-01-16,4,69.0,1,L0401,810501771,R,4,2,W,A,False,False,False,0.0,False,False,0.000,0.00,0,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10614166,10501772,2015-11-10,4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,1.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10614166,10501772,2015-11-10,4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,3.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10614166,10501772,2015-11-10,4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,2.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10614166,10501772,2015-11-10,4,88.0,1,L1001,810501772,R,1,2,W,D,True,False,False,4.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
creg_dummies = pd.get_dummies(df.CREG_CODE, prefix='CREG')
df = df.drop(columns = 'CREG_CODE')
df = pd.concat([df, creg_dummies], axis=1)
df.head()

,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,GRADE,AGE,SEX,LINKNUMBER,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,SITE_C13,SITE_C17,SITE_C20,SITE_C21,SITE_C22,SITE_C23,SITE_C24,SITE_C25,SITE_C43,SITE_C48,SITE_C50,SITE_C53,SITE_C64,SITE_C65,SITE_C70,SITE_C71,SITE_C72,SITE_C73,SITE_C78,SITE_C79,SITE_C80,SITE_C82,SITE_C91,SITE_D01,SITE_D09,SITE_D32,SITE_D33,SITE_D35,SITE_D37,SITE_D41,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR,CREG_L0201,CREG_L0301,CREG_L0401,CREG_L0801,CREG_L0901,CREG_L1001,CREG_L1201,CREG_L1701
0,10614165,10501771,2017-01-16,4,69.0,1,810501771,R,4,2,W,A,False,False,False,0.0,False,False,0.000,0.00,0,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,10614166,10501772,2015-11-10,4,88.0,1,810501772,R,1,2,W,D,True,False,False,1.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,10614166,10501772,2015-11-10,4,88.0,1,810501772,R,1,2,W,D,True,False,False,3.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,10614166,10501772,2015-11-10,4,88.0,1,810501772,R,1,2,W,D,True,False,False,2.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,10614166,10501772,2015-11-10,4,88.0,1,810501772,R,1,2,W,D,True,False,False,4.0,False,False,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [14]:
lat_dummies = pd.get_dummies(df.LATERALITY, prefix='LAT')
df = df.drop(columns = 'LATERALITY')
df = pd.concat([df, lat_dummies], axis=1)

In [15]:
eth_dummies = pd.get_dummies(df.ETHNICITY, prefix='ETH')
df = df.drop(columns = 'ETHNICITY')
df = pd.concat([df, eth_dummies], axis=1)

In [16]:
# only 80 censored
# EVENT means not alive 
# df = df.drop(df[df.NEWVITALSTATUS == 'X'].index)
vital_mapper = {'A' : 0, 'X' : 0, 'D' : 1}
df['EVENT'] = df.NEWVITALSTATUS.replace(vital_mapper)
df = df.drop(columns='NEWVITALSTATUS')


In [17]:
df.SACT = df.SACT.astype(int)
df.CLINICAL_TRIAL_INDICATOR = df.CLINICAL_TRIAL_INDICATOR.astype(int)
df.CHEMO_RADIATION_INDICATOR = df.CHEMO_RADIATION_INDICATOR.astype(int)

In [18]:
df.head()

,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,GRADE,AGE,SEX,LINKNUMBER,QUINTILE_2015,TUMOUR_COUNT,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,SITE_C13,SITE_C17,SITE_C20,SITE_C21,SITE_C22,SITE_C23,SITE_C24,SITE_C25,SITE_C43,SITE_C48,SITE_C50,SITE_C53,SITE_C64,SITE_C65,SITE_C70,SITE_C71,SITE_C72,SITE_C73,SITE_C78,SITE_C79,SITE_C80,SITE_C82,SITE_C91,SITE_D01,SITE_D09,SITE_D32,SITE_D33,SITE_D35,SITE_D37,SITE_D41,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR,CREG_L0201,CREG_L0301,CREG_L0401,CREG_L0801,CREG_L0901,CREG_L1001,CREG_L1201,CREG_L1701,LAT_9,LAT_B,LAT_L,LAT_M,LAT_R,ETH_A,ETH_B,ETH_C,ETH_M,ETH_O,ETH_U,ETH_W,EVENT
0,10614165,10501771,2017-01-16,4,69.0,1,810501771,4,2,0,False,False,0.0,0,0,0.000,0.00,0,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,10614166,10501772,2015-11-10,4,88.0,1,810501772,1,2,1,False,False,1.0,0,0,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
2,10614166,10501772,2015-11-10,4,88.0,1,810501772,1,2,1,False,False,3.0,0,0,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
3,10614166,10501772,2015-11-10,4,88.0,1,810501772,1,2,1,False,False,2.0,0,0,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
4,10614166,10501772,2015-11-10,4,88.0,1,810501772,1,2,1,False,False,4.0,0,0,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1


In [19]:
### flattening to only keep latest, most complete entry 
print(df['PATIENTID'].nunique())
to_max = ['GRADE','TUMOUR_COUNT','SACT','REGIMEN_COUNT','CLINICAL_TRIAL_INDICATOR','CHEMO_RADIATION_INDICATOR','DAYS_TO_FIRST_SURGERY','DAYS_SINCE_DIAGNOSIS']
to_max = to_max + list(df.columns[df.columns.str.startswith('SITE_') | df.columns.str.startswith('LAT_')])
to_median = ['QUINTILE_2015']
to_min = ['AGE']

for m in to_max:
    df = add_aggregate(df,what='max',target=m,groupby='PATIENTID',name=m)
for m in to_median:
    df = add_aggregate(df,what='median',target=m,groupby='PATIENTID',name=m)
for m in to_min:
    df = add_aggregate(df,what='min',target=m,groupby='PATIENTID',name=m)
df.head()

39940
lost:  0
lost:  0
lost:  0
lost:  0
lost:  9107
lost:  484
lost:  564
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0
lost:  0


,TUMOURID,PATIENTID,DIAGNOSISDATEBEST,GRADE,AGE,SEX,LINKNUMBER,QUINTILE_2015,TUMOUR_COUNT,SACT,CLINICAL_TRIAL,CHEMO_RADIATION,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,SITE_C13,SITE_C17,SITE_C20,SITE_C21,SITE_C22,SITE_C23,SITE_C24,SITE_C25,SITE_C43,SITE_C48,SITE_C50,SITE_C53,SITE_C64,SITE_C65,SITE_C70,SITE_C71,SITE_C72,SITE_C73,SITE_C78,SITE_C79,SITE_C80,SITE_C82,SITE_C91,SITE_D01,SITE_D09,SITE_D32,SITE_D33,SITE_D35,SITE_D37,SITE_D41,SITE_D43,SITE_D47,SITE_C44,SITE_C51,SITE_C54,SITE_C67,SITE_C77,SITE_C84,SITE_C92,SITE_D03,BENIGN_BEHAVIOUR,CREG_L0201,CREG_L0301,CREG_L0401,CREG_L0801,CREG_L0901,CREG_L1001,CREG_L1201,CREG_L1701,LAT_9,LAT_B,LAT_L,LAT_M,LAT_R,ETH_A,ETH_B,ETH_C,ETH_M,ETH_O,ETH_U,ETH_W,EVENT
0,10614165,10501771,2017-01-16,4,69.0,1,810501771,4.0,2,0,False,False,0.0,0,0,0.000,0.00,0,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,10614166,10501772,2015-11-10,4,88.0,1,810501772,1.0,2,1,False,False,5.0,0,0,1.755,71.75,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
6,10614174,10501888,2017-04-27,4,79.0,1,810501888,2.0,2,0,False,False,0.0,0,0,0.000,0.00,0,252,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
7,10614177,10501946,2016-12-23,-1,79.0,2,810501946,2.0,2,0,False,False,0.0,0,0,0.000,0.00,15,425,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1
8,10614180,10501963,2014-01-02,-1,57.0,1,810501963,3.0,2,0,False,False,0.0,0,0,0.000,0.00,15,1864,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [20]:
df['PATIENTID'].nunique()

39940

In [21]:
lost = set(patient_ids) - set(df.PATIENTID)
print(lost)

{170010625, 190004737, 140013571, 160018440, 250012177, 160013844, 300012564, 210018344, 10560040, 290023464, 260019241, 210007597, 180021293, 280021042, 180020788, 330010167, 300009527, 210013757, 250017346, 250018883, 140021316, 100037194, 300010061, 270004318, 80029793, 180021349, 250020970, 80030317, 320024685, 330024561, 30185074, 300019828, 300010101, 30178423, 80040069, 210014855, 210009225, 300012170, 250018444, 210000013, 300010130, 210020506, 170012831, 210004128, 140019875, 210017955, 250013859, 50085032, 300006572, 250016432, 140013746, 80034485, 160025784, 180023486, 300011711, 300013246, 210010819, 250013891, 140033735, 210019528, 140032712, 250012880, 170010323, 50102489, 240022234, 140032732, 140026590, 150029553, 150026485, 40146680, 180020474, 140002562, 210001156, 210003208, 330018060, 330015516, 330010409, 80031025, 300011318, 170015545, 250022203, 210009404, 140021054, 140012351, 50087236, 230018373, 210017610, 250014026, 300009807, 80033112, 260014936, 250018138, 

In [22]:
# unscaled pandas 
print(len(df))
to_csv(df, './Data/data.csv', index=False)

# this would give headerless numpy 
# data = scale(df)
# np.savetxt("./Data/data.csv", data, delimiter=",")

42622


In [23]:
pd.set_option('display.max_rows', None)
df.dtypes


TUMOURID                              int64
PATIENTID                             int64
DIAGNOSISDATEBEST            datetime64[ns]
GRADE                                 int64
AGE                                 float64
SEX                                category
LINKNUMBER                            int64
QUINTILE_2015                       float64
TUMOUR_COUNT                          int64
SACT                                  int64
CLINICAL_TRIAL                         bool
CHEMO_RADIATION                        bool
REGIMEN_COUNT                       float64
CLINICAL_TRIAL_INDICATOR              int64
CHEMO_RADIATION_INDICATOR             int64
NORMALISED_HEIGHT                   float64
NORMALISED_WEIGHT                   float64
DAYS_TO_FIRST_SURGERY                 int64
DAYS_SINCE_DIAGNOSIS                  int64
SITE_C13                              uint8
SITE_C17                              uint8
SITE_C20                              uint8
SITE_C21                        